In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
customer = pd.read_excel('/content/drive/MyDrive/dataset/CapstoneProject_Dataset_2021.xlsx', sheet_name='CustomerProfile')
sale = pd.read_excel('/content/drive/MyDrive/dataset/CapstoneProject_Dataset_2021.xlsx', sheet_name='Sale')

#KIỂM TRA DỮ LIỆU


In [4]:
missing_data_sheet2 = sale.isnull().sum()
print("\nDữ liệu bị thiếu trong Sheet 2:")
print(missing_data_sheet2)


Dữ liệu bị thiếu trong Sheet 2:
CustomerID     3
OrderID        7
Channel        3
PaymentType    2
Brand          2
Shop           5
Promotion      0
SalePrice      7
SaleDate       0
Reference      5
dtype: int64


#XỮ LÝ DỮ LIỆU


XỮ LÝ DỮ LIỆU CỦA CỘT SALEPRICE


In [5]:
# KIỂM TRA LOẠI DỮ LIỆU GÌ
sale.SalePrice
dtype_B = sale['SalePrice'].dtype
print("Data type of column 'B':", dtype_B)

Data type of column 'B': object


In [8]:
# HÀM BIẾN ĐỔI TEXT THÀNH NUMBER VÀ CHỈNH SỬA ĐỐI VỚI CÁC KÝ TỰ ĐẶC BIỆT
def convert_to_number(value):
    if isinstance(value, str) and value.startswith('$'):
        value = float(value.replace('$', '')) * 25000
    return value

In [9]:
sale['SalePrice'] = sale['SalePrice'].apply(convert_to_number)

In [11]:
sale['SalePrice']= sale['SalePrice'].astype(float)
sale['SalePrice']

0       27000000.0
1        3020000.0
2       18310000.0
3       12203000.0
4       13070000.0
           ...    
1561     4248000.0
1562    14390000.0
1563           NaN
1564    18600000.0
1565    41025000.0
Name: SalePrice, Length: 1566, dtype: float64

In [12]:
# THỰC HIỆN BIẾN ĐỔI USA SANG VND
def multiply_if(value):
    if value < 5000:
        return value * 25000
    else:
        return value


sale['SalePrice'] = sale['SalePrice'].apply(multiply_if)

XỬ DỤNG ONE-HOT INCODING CHO CỘT REFERENCE


In [14]:
from sklearn.preprocessing import MultiLabelBinarizer
sale['Reference'] = sale['Reference'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [15]:
mlb = MultiLabelBinarizer()
one_hot_genres = mlb.fit_transform(sale['Reference'])

In [16]:
genres_df = pd.DataFrame(one_hot_genres, columns=mlb.classes_)
SALE = pd.concat([sale, genres_df], axis=1)

In [17]:
SALE.head()

,CustomerID,OrderID,Channel,PaymentType,Brand,Shop,Promotion,SalePrice,SaleDate,Reference,Khác,Mạng xã hội,Nhân viên cửa hàng bán Laptop,Nhờ người có chuyên môn tư vấn,Phương tiện truyền thông (báo,Thông qua người thân,Website,bạn bè,"tivi,...)"
0,CTM012,OTS015,[Online],Thẻ,LenovoX1,Phong Vũ,Không,27000000.0,2020-04-23 00:00:00,[],0,0,0,0,0,0,0,0,0
1,CTM10164,OTS831,[Offline],Tiền mặt,Apple,Viettel Store,Không,3020000.0,2020-08-03 00:00:00,[Nhân viên cửa hàng bán Laptop],0,0,1,0,0,0,0,0,0
2,CTM10246,OTS293,[Offline],Tiền mặt,Dell,FPT Shop,Có,18310000.0,2020-10-09 00:00:00,"[Nhân viên cửa hàng bán Laptop, Mạng xã hội, W...",0,1,1,0,0,0,1,0,0
3,CTM1028,OTS34756,[Offline],Tiền mặt,MSI,Phong Vũ,Không,12203000.0,2020-05-24 00:00:00,"[Thông qua người thân, bạn bè, Nhân viên cửa h...",0,1,1,0,0,1,0,1,0
4,CTM1028,OTS1044,[Offline],Tiền mặt,Asus,Thế giới di động,Có,13070000.0,2020-10-08 00:00:00,[Nhờ người có chuyên môn tư vấn],0,0,0,1,0,0,0,0,0


#XŨ LÝ BẲNG CUSTOMER


In [19]:
from sklearn.preprocessing import MultiLabelBinarizer
customer['MucDichSuDungLaptop'] = customer['MucDichSuDungLaptop'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [20]:
mlb = MultiLabelBinarizer()
one_hot_genres = mlb.fit_transform(customer['MucDichSuDungLaptop'])

In [21]:
genres = pd.DataFrame(one_hot_genres, columns=mlb.classes_)
Customer = pd.concat([customer, genres], axis=1)

In [22]:
Customer.head()

,KH_ID,GioiTinh,QueQuan,NamHocThu,ChuyenNganh,NgheNghiepGiaDinh,ChiTieuHangThang,ThoiGianSuDung,MucDichSuDungLaptop,MucDoHaiLong,...,YeuToBaoMat,YeuToBaoHanh,YeuToKhuyenMai,Chơi game,Giải trí (xem phim,Học tập,Khác,Làm việc,lướt web),nghe nhạc
0,CTM10164,Nam,Quảng Trị,Năm 2,Luật kinh tế,Nông dân,Từ 1.500.000 đến 3.000.000,Từ 6 tháng - 1 năm,"[Học tập, Làm việc, Chơi game, Giải trí (xem p...",5,...,3,3,3,1,1,1,1,1,1,1
1,CTM10246,Nam,Đà Nẵng,Năm 4,Hệ thống thông tin quản lý,Kinh doanh tự do,Từ 3.000.000 đến 5.000.000,Trên 2 năm,"[Học tập, Làm việc, Chơi game]",4,...,1,5,5,1,0,1,0,1,0,0
2,CTM1028,Nữ,Nghệ An,Năm 3,Thương mại điện tử,Nông dân,Từ 1.500.000 đến 3.000.000,Trên 2 năm,"[Học tập, Làm việc]",4,...,2,4,4,0,0,1,0,1,0,0
3,CTM10516,Nam,Gia Lai,Năm 2,Thương mại,Nông dân,Từ 1.500.000 đến 3.000.000,Từ 1 năm đến 2 năm,"[Học tập, Làm việc, Giải trí (xem phim, nghe n...",4,...,3,4,2,0,1,1,0,1,1,1
4,CTM10573,Nam,Thừa Thiên Huế,Năm 4,Tài chính doanh nghiệp,Nông dân,Từ 1.500.000 đến 3.000.000,Trên 2 năm,"[Học tập, Làm việc]",3,...,1,3,4,0,0,1,0,1,0,0
